# Homework 3

I have a catalog of repeating earthquakes, and I want to download seismic waveforms corresponding to these repeating earthquakes. However, when I look at the list of stations available in the seismic network, there are more than 6000. I do not want to download data from 6000 stations, so I want to filter only the seismic stations that are relevant for what I want to do with my waveforms.

In [ ]:
# Address of the website to download data
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

In [ ]:
# Useful Python modules
import numpy as np
import pandas as pd
import io
import pickle
import requests
from datetime import datetime, timedelta
from math import cos, sin, pi, sqrt

In [ ]:
# Import the data from the website into a Python dataframe
s = requests.get(url).content
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

In [ ]:
# Transform columns start_time and end_time into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

After discussing with my adviser, we decided than only the following channels are relevant for the work we want to do:

In [ ]:
channels = ['BHE', 'BHN', 'BHZ', 'BH1', 'BH2', \
            'EHE', 'EHN', 'EHZ', 'EH1', 'EH2', \
            'HHE', 'HHN', 'HHZ', 'HH1', 'HH2', \
            'SHE', 'SHN', 'SHZ', 'SH1', 'SH2']

## First question

Filter the dataset to keep only the rows with the channels as defined above.

My earthquake catalog starts on 2007/07/01 and ends on 2009/07/01. I am only interested in stations that started recording before 2007/07/01 and ended recording after 2009/07/01.

## Second question

Filter the dataset to keep only stations that started recording before 2007/07/01 and ended recording after 2009/07/01.

I only want to keep the stations that are located less than 100 km from my repeating earthquakes. For stations farther away, the signal-to-noise ratio would be too low.

The earthquakes are located at latitude = 40.09 and longitude = -122.87. Here is a function to compute the distance from the station to the earthquakes, and to add a column distance to the dataset

In [ ]:
a = 6378.136
e = 0.006694470
lat0 = 40.09000
lon0 = -122.87000
dx = (pi / 180.0) * a * cos(lat0 * pi / 180.0) / sqrt(1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0))
dy = (3.6 * pi / 648.0) * a * (1.0 - e * e) / ((1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0)) ** 1.5)
x = dx * (data['longitude'] - lon0)
y = dy * (data['latitude'] - lat0)
data['distance'] = np.sqrt(np.power(x, 2.0) + np.power(y, 2.0))

## Third question

Filter the dataset to keep only stations that are less than 100 km from the earthquakes.

Finally, I want to group the result such that the final result looks like:

|station|network|location|latitude|longitude |elevation|depth|distance |channel    |start_time         |end_time           |
|-------|-------|--------|--------|----------|---------|-----|---------|-----------|-------------------|-------------------|
|KBS 	|NC 	|-- 	 |39.91719|-123.59561|1120.0   |0.0  |64.720762|SHZ        |2002-10-17 00:00:00|2011-10-27 21:25:00|
|KCPB 	|NC 	|-- 	 |39.68631|-123.58242|1261.0   |0.0  |75.502041|HHZ,HHN,HHE|2006-10-18 00:08:00|2010-11-01 22:00:00|

I want all different channels to be grouped together, instead of having one row per channel. I also want to get the start_time end end_time for each station, instead of having it for each channel.

You can use the following function to group the channels together:

In [ ]:
def f(x):
    """
    Concatenate channels
    """
    result = '%s' % ','.join(x)
    result = list(set(result.split(',')))
    result = '%s' % ','.join(result)
    return result

## Fourth question

Use the pandas function agg to group the channels of a given station together, and compute the least recent start_time and the most recent end_time for each station.

## Fifth question

How many stations are left in the dataset?